In [19]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [47]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1720948382481, experiment_id='1', last_update_time=1720948382481, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [22]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [23]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [24]:
len(df_train), len(df_val)

(73908, 61921)

In [25]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [26]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [27]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715209663881

In [29]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [30]:
with mlflow.start_run():

    mlflow.set_tag("developer", "tejash")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
!pip install xgboost

     |████████████████████████████████| 153.9 MB 54 kB/s s eta 0:00:01     |████████████████████████▌       | 118.0 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 190.9 MB 21 kB/s s eta 0:00:01    |███████████                     | 65.4 MB 99.4 MB/s eta 0:00:02


In [31]:
import xgboost as xgb

In [32]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [33]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [34]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [35]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                                              | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:22:05] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.26655                                                                                                                              
[1]	validation-rmse:8.96611                                                                                                                               
[2]	validation-rmse:8.12245                                                                                                                               
[3]	validation-rmse:7.58679                                                                                                                               
[4]	validation-rmse:7.25162                                                                                                                               
[5]	validation-rmse:7.04036                                                                                                                               
[6]	validation-rmse:6.90531                                           

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:23:59] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.82318                                                                                                                              
[1]	validation-rmse:9.75498                                                                                                                               
[2]	validation-rmse:8.94480                                                                                                                               
[3]	validation-rmse:8.33863                                                                                                                               
[4]	validation-rmse:7.88938                                                                                                                               
[5]	validation-rmse:7.56018                                                                                                                               
[6]	validation-rmse:7.31745                                           

...

[111]	validation-rmse:5.43209                                                                                                                             
[112]	validation-rmse:5.43041                                                                                                                             
[113]	validation-rmse:5.43027                                                                                                                             
[114]	validation-rmse:5.42865                                                                                                                             
[115]	validation-rmse:5.42800                                                                                                                             
[116]	validation-rmse:5.42690                                                                                                                             
[117]	validation-rmse:5.42643                                         

[215]	validation-rmse:5.37699                                                                                                                             
[216]	validation-rmse:5.37633                                                                                                                             
[217]	validation-rmse:5.37587                                                                                                                             
[218]	validation-rmse:5.37405                                                                                                                             
[219]	validation-rmse:5.37462                                                                                                                             
[220]	validation-rmse:5.37434                                                                                                                             
[221]	validation-rmse:5.37563                                         

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:09:29] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.46830                                                                                                                               
[1]	validation-rmse:5.64405                                                                                                                               
[2]	validation-rmse:5.40790                                                                                                                               
[3]	validation-rmse:5.30724                                                                                                                               
[4]	validation-rmse:5.27795                                                                                                                               
[5]	validation-rmse:5.26793                                                                                                                               
[6]	validation-rmse:5.26348                                           

[104]	validation-rmse:5.16265                                                                                                                             
[105]	validation-rmse:5.16203                                                                                                                             
[106]	validation-rmse:5.16192                                                                                                                             
[107]	validation-rmse:5.16256                                                                                                                             
[108]	validation-rmse:5.16074                                                                                                                             
[109]	validation-rmse:5.16152                                                                                                                             
[110]	validation-rmse:5.16258                                         

In [39]:
with mlflow.start_run():
    
    best_params = {
        'learning_rate' : 0.22259759382341754,
        'max_depth' : 36,
        'min_child_weight' : 1.2409767785922285,
        'objective' : 'reg:linear',
        'reg_alpha' : 0.08900594938257661,
        'reg_lambda' : 0.16431341133360675,
        'seed' : 42
    }
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster,artifact_path="modles_mlflow")

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:32:53] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.46355
[1]	validation-rmse:9.21840
[2]	validation-rmse:8.35743
[3]	validation-rmse:7.76696
[4]	validation-rmse:7.37597
[5]	validation-rmse:7.11050
[6]	validation-rmse:6.93345
[7]	validation-rmse:6.81225
[8]	validation-rmse:6.72940
[9]	validation-rmse:6.66908
[10]	validation-rmse:6.62538
[11]	validation-rmse:6.59261
[12]	validation-rmse:6.56866
[13]	validation-rmse:6.55180
[14]	validation-rmse:6.53595
[15]	validation-rmse:6.52559
[16]	validation-rmse:6.51772
[17]	validation-rmse:6.51068
[18]	validation-rmse:6.50547
[19]	validation-rmse:6.50085
[20]	validation-rmse:6.49867
[21]	validation-rmse:6.49733
[22]	validation-rmse:6.49432
[23]	validation-rmse:6.49261
[24]	validation-rmse:6.49026
[25]	validation-rmse:6.48830
[26]	validation-rmse:6.48616
[27]	validation-rmse:6.48406
[28]	validation-rmse:6.48223
[29]	validation-rmse:6.48013
[30]	validation-rmse:6.47741
[31]	validation-rmse:6.47546
[32]	validation-rmse:6.47296
[33]	validation-rmse:6.47153
[34]	validation-rmse:6.

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:34:50] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [57]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import mlflow.sklearn

# mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.sklearn.log_model(mlmodel,artifact_path="modles_mlflow")
        mlflow.log_metric("rmse", rmse)

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is rep

In [58]:
# Predict on a Pandas DataFrame:

import mlflow
logged_model = 'runs:/837733a73ab040faaa4ecff356fc5283/modles_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [59]:
pred = loaded_model.predict(X_val)

In [60]:
pred[0:10]

array([14.094945 ,  6.9943886, 14.723225 , 24.52018  ,  9.4468355,
       17.175951 , 10.459017 ,  8.143998 , 10.084587 , 18.48966  ],
      dtype=float32)

In [61]:
y_val[0:10]

array([17.91666667,  6.5       , 15.25      , 18.23333333,  8.96666667,
        7.85      ,  9.7       , 11.28333333,  8.73333333,  1.71666667])